pca_fasttext_50features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings("ignore") 

In [4]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [5]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [7]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [9]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [11]:

df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [12]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/fasttext_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-6.479078,1.651257,0.896314,3.535852,2.867422,1.560191,-0.960358,-4.078380,0.547516,...,6.695309e-07,0.000003,9.046899e-08,-4.097595e-07,-4.108224e-07,-0.000002,1.357596e-06,1.974855e-07,-1.126599e-06,-3.217740e-06
1,1,-5.452967,2.308225,-6.763821,4.569993,0.856911,-0.518583,-2.731985,-0.432451,0.335011,...,3.345125e-06,0.000006,8.161220e-07,-1.593020e-07,3.119997e-06,-0.000002,-8.159281e-07,-4.400613e-07,3.657790e-06,6.288601e-07
2,2,-8.162915,0.634287,-1.216198,-1.509451,-1.288995,3.823911,-1.172312,0.457859,-2.033172,...,-1.055074e-05,-0.000003,4.170370e-06,6.558600e-06,1.725653e-06,-0.000003,-1.960362e-06,-3.647499e-06,1.803280e-07,6.976526e-07
3,3,-6.632410,3.739571,-1.834406,0.031421,0.661350,-1.142883,-3.133973,-1.202540,-2.633310,...,7.818613e-06,0.000005,2.075646e-06,2.829128e-06,1.148339e-06,0.000002,-1.569741e-06,-3.400200e-07,2.084161e-06,4.895014e-07
4,4,-1.823112,2.290021,3.397779,5.673997,-2.436430,-2.122263,-0.676238,7.185288,-0.713091,...,-2.626606e-06,-0.000004,-4.484433e-06,3.324942e-06,-1.525081e-06,0.000002,1.449589e-06,-1.708901e-06,4.329716e-07,-1.685343e-06


In [13]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-6.479078,1.651257,0.896314,3.535852,2.867422,1.560191,-0.960358,-4.078380,0.547516,-1.175984,...,6.695309e-07,0.000003,9.046899e-08,-4.097595e-07,-4.108224e-07,-0.000002,1.357596e-06,1.974855e-07,-1.126599e-06,-3.217740e-06
1,-5.452967,2.308225,-6.763821,4.569993,0.856911,-0.518583,-2.731985,-0.432451,0.335011,-1.138042,...,3.345125e-06,0.000006,8.161220e-07,-1.593020e-07,3.119997e-06,-0.000002,-8.159281e-07,-4.400613e-07,3.657790e-06,6.288601e-07
2,-8.162915,0.634287,-1.216198,-1.509451,-1.288995,3.823911,-1.172312,0.457859,-2.033172,-0.040681,...,-1.055074e-05,-0.000003,4.170370e-06,6.558600e-06,1.725653e-06,-0.000003,-1.960362e-06,-3.647499e-06,1.803280e-07,6.976526e-07
3,-6.632410,3.739571,-1.834406,0.031421,0.661350,-1.142883,-3.133973,-1.202540,-2.633310,4.376703,...,7.818613e-06,0.000005,2.075646e-06,2.829128e-06,1.148339e-06,0.000002,-1.569741e-06,-3.400200e-07,2.084161e-06,4.895014e-07
4,-1.823112,2.290021,3.397779,5.673997,-2.436430,-2.122263,-0.676238,7.185288,-0.713091,1.961139,...,-2.626606e-06,-0.000004,-4.484433e-06,3.324942e-06,-1.525081e-06,0.000002,1.449589e-06,-1.708901e-06,4.329716e-07,-1.685343e-06


In [14]:
y=df['Level '] 

**Train Test Split**

In [15]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [16]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3400710993036219


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.52541,0.52541,0.572603,0.528953,0.326336,0.326336,0.374192,0.332476


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3115    0.1210    0.1743       157
           2     0.4570    0.8523    0.5950       237
           3     0.3810    0.0615    0.1060       130

    accuracy                         0.4370       524
   macro avg     0.3831    0.3450    0.2918       524
weighted avg     0.3945    0.4370    0.3476       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.52541,0.52541,0.572603,0.528953,0.326336,0.326336,0.374192,0.332476
1,RandomForest,1.00000,1.00000,1.000000,1.000000,0.437023,0.437023,0.394538,0.347624


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5181    0.3961    0.4490       361
           2     0.5595    0.7649    0.6463       553
           3     0.4894    0.3007    0.3725       306

    accuracy                         0.5393      1220
   macro avg     0.5223    0.4872    0.4892      1220
weighted avg     0.5297    0.5393    0.5192      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3120    0.2484    0.2766       157
           2     0.4582    0.5781    0.5112       237
           3     0.2000    0.1538    0.1739       130

    accuracy                         0.3740       524
   macro avg     0.3234    0.3268    0.3206       524
weighted avg     0.3503    0.3740    0.3572       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.543532,0.525899,0.362595,0.362595,0.342859,0.344120
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.370898,0.354064
2,Adaboost,0.539344,0.539344,0.529673,0.519228,0.374046,0.374046,0.350336,0.357228


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9806    0.8421    0.9061       361
           2     0.8389    0.9982    0.9116       553
           3     0.9841    0.8105    0.8889       306

    accuracy                         0.9049      1220
   macro avg     0.9346    0.8836    0.9022      1220
weighted avg     0.9173    0.9049    0.9043      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3119    0.2166    0.2556       157
           2     0.4409    0.6456    0.5240       237
           3     0.2059    0.1077    0.1414       130

    accuracy                         0.3836       524
   macro avg     0.3196    0.3233    0.3070       524
weighted avg     0.3440    0.3836    0.3487       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.543532,0.525899,0.362595,0.362595,0.342859,0.344120
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.370898,0.354064
2,Adaboost,0.539344,0.539344,0.529673,0.519228,0.374046,0.374046,0.350336,0.357228
3,GBM,0.904918,0.904918,0.917271,0.904299,0.383588,0.383588,0.343961,0.348665


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3897    0.4598    0.4219       361
           2     0.5643    0.4521    0.5020       553
           3     0.3618    0.4150    0.3866       306

    accuracy                         0.4451      1220
   macro avg     0.4386    0.4423    0.4368      1220
weighted avg     0.4619    0.4451    0.4493      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3240    0.3694    0.3452       157
           2     0.4479    0.3629    0.4009       237
           3     0.2026    0.2385    0.2191       130

    accuracy                         0.3340       524
   macro avg     0.3249    0.3236    0.3218       524
weighted avg     0.3499    0.3340    0.3391       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.525410,0.525410,0.572603,0.528953,0.326336,0.326336,0.374192,0.332476
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.437023,0.437023,0.394538,0.347624
2,SVM,0.445082,0.445082,0.461857,0.449346,0.333969,0.333969,0.349938,0.339130


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4677    0.5208    0.4928       361
           2     0.5651    0.7143    0.6310       553
           3     0.4874    0.1895    0.2729       306

    accuracy                         0.5254      1220
   macro avg     0.5067    0.4749    0.4656      1220
weighted avg     0.5168    0.5254    0.5003      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2984    0.3631    0.3276       157
           2     0.4351    0.5232    0.4751       237
           3     0.1042    0.0385    0.0562       130

    accuracy                         0.3550       524
   macro avg     0.2792    0.3082    0.2863       524
weighted avg     0.3120    0.3550    0.3270       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.543532,0.525899,0.362595,0.362595,0.342859,0.344120
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.370898,0.354064
2,Adaboost,0.539344,0.539344,0.529673,0.519228,0.374046,0.374046,0.350336,0.357228
3,GBM,0.904918,0.904918,0.917271,0.904299,0.383588,0.383588,0.343961,0.348665
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.525410,0.525410,0.516775,0.500291,0.354962,0.354962,0.312044,0.326970


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3220    0.6842    0.4379       361
           2     0.5284    0.2188    0.3095       553
           3     0.3750    0.2745    0.3170       306

    accuracy                         0.3705      1220
   macro avg     0.4085    0.3925    0.3548      1220
weighted avg     0.4289    0.3705    0.3494      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3072    0.6497    0.4172       157
           2     0.4433    0.1814    0.2575       237
           3     0.2211    0.1615    0.1867       130

    accuracy                         0.3168       524
   macro avg     0.3239    0.3309    0.2871       524
weighted avg     0.3474    0.3168    0.2878       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.543532,0.525899,0.362595,0.362595,0.342859,0.344120
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.370898,0.354064
2,Adaboost,0.539344,0.539344,0.529673,0.519228,0.374046,0.374046,0.350336,0.357228
3,GBM,0.904918,0.904918,0.917271,0.904299,0.383588,0.383588,0.343961,0.348665
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.525410,0.525410,0.516775,0.500291,0.354962,0.354962,0.312044,0.326970
6,GNB,0.370492,0.370492,0.428853,0.349366,0.316794,0.316794,0.347392,0.287763
